In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Roraima - Produção de Cimento (t)'], axis=1)
data

,Ano,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Consumo de Cimento (t)
0,2003,8.263084,3.596068,0.734916,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,1670.287074,25.596646,22.710362,82.264047,1.617738,6.128460e+06,332580.202821,12.101540,5.339136e+06,74.283000
1,2004,8.191545,3.429895,0.735404,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,1661.128565,25.697278,22.602565,81.755854,1.624959,6.226406e+06,336594.835486,12.124873,5.374190e+06,39.775000
2,2005,8.166961,3.071663,0.735800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,1643.063802,25.858925,22.427865,80.748384,1.635661,6.321383e+06,339978.313869,12.165244,5.409179e+06,30.179000
3,2006,8.072306,3.045251,0.736105,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,1618.868877,26.070821,22.210663,79.369072,1.651218,6.410128e+06,343100.125013,12.200109,5.439894e+06,34.243000
4,2007,8.047939,3.353816,0.736322,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,1591.319700,26.322200,21.975359,77.745355,1.673006,6.486417e+06,340068.183172,12.212555,5.458573e+06,35.931000
5,2008,8.015090,3.689138,0.736454,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,1563.192183,26.602295,21.746354,76.004671,1.702399,6.555833e+06,342849.112404,12.235418,5.474033e+06,60.009000
6,2009,8.046321,3.874501,0.736503,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,1537.262238,26.900339,21.548047,74.274459,1.740774,6.611657e+06,345335.485288,12.228462,5.482306e+06,63.064000
7,2010,8.009229,4.024163,0.736471,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,1516.305777,27.205566,21.404841,72.682157,1.789506,6.787712e+06,348962.483930,12.215749,5.534487e+06,86.408000
8,2011,7.996819,4.171630,0.736360,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,1506.456126,27.425647,21.330889,71.595088,1.830447,6.947966e+06,347630.972013,12.206783,5.574553e+06,103.009000
9,2012,8.651837,4.318665,0.736089,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,1515.527356,27.608612,21.117524,70.061650,1.898255,7.096137e+06,347411.232273,12.186539,5.609510e+06,100.066000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,8.263084,3.596068,0.734916,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1001.078442,1670.287074,25.596646,22.710362,82.264047,1.617738,6.128460e+06,332580.202821,12.101540,5.339136e+06
1,8.191545,3.429895,0.735404,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1001.049675,1661.128565,25.697278,22.602565,81.755854,1.624959,6.226406e+06,336594.835486,12.124873,5.374190e+06
2,8.166961,3.071663,0.735800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1000.992392,1643.063802,25.858925,22.427865,80.748384,1.635661,6.321383e+06,339978.313869,12.165244,5.409179e+06
3,8.072306,3.045251,0.736105,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1000.933928,1618.868877,26.070821,22.210663,79.369072,1.651218,6.410128e+06,343100.125013,12.200109,5.439894e+06
4,8.047939,3.353816,0.736322,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1000.901620,1591.319700,26.322200,21.975359,77.745355,1.673006,6.486417e+06,340068.183172,12.212555,5.458573e+06
5,8.015090,3.689138,0.736454,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1000.922803,1563.192183,26.602295,21.746354,76.004671,1.702399,6.555833e+06,342849.112404,12.235418,5.474033e+06
6,8.046321,3.874501,0.736503,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1001.024814,1537.262238,26.900339,21.548047,74.274459,1.740774,6.611657e+06,345335.485288,12.228462,5.482306e+06
7,8.009229,4.024163,0.736471,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1001.234988,1516.305777,27.205566,21.404841,72.682157,1.789506,6.787712e+06,348962.483930,12.215749,5.534487e+06
8,7.996819,4.171630,0.736360,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447,6.947966e+06,347630.972013,12.206783,5.574553e+06
9,8.651837,4.318665,0.736089,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255,7.096137e+06,347411.232273,12.186539,5.609510e+06


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      39.775000
1      30.179000
2      34.243000
3      35.931000
4      60.009000
5      63.064000
6      86.408000
7     103.009000
8     100.066000
9     121.914000
10    120.417571
11    113.282057
12    105.481371
13     99.723000
14     83.012000
15    121.120000
16    151.571000
17    144.989000
18    132.217000
19           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,-0.709627,-0.806714,-0.174694,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,-0.863762,1.341889,-1.697608,1.755771,1.650951,-1.465708,-1.590661,-2.240264,-1.209834,-1.737333
1,-0.771439,-1.021390,0.118862,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,-0.924765,1.249668,-1.585990,1.613615,1.564199,-1.409306,-1.395560,-1.319744,-0.740358,-1.464888
2,-0.792681,-1.484184,0.356742,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,-1.046240,1.067766,-1.406695,1.383233,1.392215,-1.325725,-1.206374,-0.543941,0.071913,-1.192940
3,-0.874466,-1.518306,0.540358,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,-1.170217,0.824138,-1.171666,1.096801,1.156756,-1.204224,-1.029601,0.171864,0.773399,-0.954220
4,-0.895520,-1.119675,0.671120,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,-1.238729,0.546734,-0.892844,0.786498,0.879574,-1.034062,-0.877638,-0.523335,1.023829,-0.809039
5,-0.923903,-0.686479,0.750440,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,-1.193808,0.263506,-0.582171,0.484501,0.582425,-0.804497,-0.739368,0.114309,1.483848,-0.688880
6,-0.896919,-0.447012,0.779730,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-0.977484,0.002407,-0.251589,0.222987,0.287064,-0.504788,-0.628171,0.684413,1.343888,-0.624586
7,-0.928968,-0.253666,0.760401,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-0.531791,-0.208612,0.086959,0.034136,0.015245,-0.124193,-0.277482,1.516052,1.088083,-0.219018
8,-0.939690,-0.063156,0.693865,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,0.117558,-0.307792,0.331066,-0.063387,-0.170326,0.195562,0.041732,1.210748,0.907693,0.092385
9,-0.373730,0.126796,0.530420,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,0.433829,-0.216450,0.534006,-0.344758,-0.432096,0.725143,0.336878,1.160364,0.500379,0.364075


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      39.775000
1      30.179000
2      34.243000
3      35.931000
4      60.009000
5      63.064000
6      86.408000
7     103.009000
8     100.066000
9     121.914000
10    120.417571
11    113.282057
12    105.481371
13     99.723000
14     83.012000
15    121.120000
16    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
17,0.674508,2.184119,-2.88705,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.268189,-3.467872,1.896178,-1.613363,-1.878166,2.172551,0.943905,-0.349474,-1.517602,0.929344


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
14,1.252131,1.123678,-0.943913,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,1.420842,-0.416990,0.821687,-1.096409,-0.971191,1.076035,1.261385,-0.537740,-0.929289,1.250944
15,1.346017,1.319327,-1.619925,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,1.480325,-0.687775,0.770285,-1.055336,-0.916375,0.837768,1.271596,-0.450795,-1.442269,1.273228
16,1.162902,1.813945,-3.122943,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,0.552645,-3.249208,1.442333,-1.310640,-1.425237,1.203139,1.180119,-0.498884,-1.686584,1.168121


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[512351715, 1306222424, 354761571, 63978127, 2804061436, 2479482017, 2213154325, 1369917768, 2389017532, 923169991, 3940021771, 4024101861, 3248001488, 2566666606, 181983459, 8169112, 1556663755, 2456088311, 1779516608, 703230068, 3934661564, 2010727462, 2168154291, 1187123697, 2372845329, 525557720, 1216188789, 207753009, 1990541945, 1601779876, 2470495775, 3475987750, 579540791, 1566118875, 3694483885, 177089763, 2891500113, 957679804, 3624133168, 2647291299, 1239533335, 1475465557, 1087207722, 4177445091, 4002582111, 2598711068, 2631594638, 2792772517, 3918609084, 3214236890, 1956619957, 2288641252, 1734556231, 1367718725, 901203564, 2284720188, 1674834666, 1483482826, 2131580942, 3362823422, 3801888117, 1646299674, 4222373592, 1857452191, 1203929621, 1454547360, 1854655375, 3294394381, 3282729604, 1822490242, 4273281977, 1275891269, 4013248128, 1169095733, 3400535482, 3946680495, 2229581233, 2956063687, 992664632, 3864681248, 271239680, 2879072551, 3467078843, 2893894527, 353337483

val_loss: 813.032958984375


Step: 84 ___________________________________________
val_loss: 229.760498046875


Step: 85 ___________________________________________
val_loss: 268.3167724609375


Step: 86 ___________________________________________
val_loss: 263.2221984863281


Step: 87 ___________________________________________
val_loss: 12.968390464782715


Step: 88 ___________________________________________
val_loss: 757.403076171875


Step: 89 ___________________________________________
val_loss: 830.3168334960938


Step: 90 ___________________________________________
val_loss: 0.17383652925491333


Step: 91 ___________________________________________
val_loss: 777.882568359375


Step: 92 ___________________________________________
val_loss: 799.40966796875


Step: 93 ___________________________________________
val_loss: 4.7116889618337154e-05
winner_seed: 2509986990


Step: 94 ___________________________________________
val_loss: 27.15535545349121


Step: 95 ______________________

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 798ms/step - loss: 8742.4414 - val_loss: 12047.6201
Epoch 2/10000
1/1 [==============================] - 0s 17ms/step - loss: 7647.8062 - val_loss: 12392.8271
Epoch 3/10000
1/1 [==============================] - 0s 16ms/step - loss: 5285.1538 - val_loss: 3113.8535
Epoch 4/10000
1/1 [==============================] - 0s 17ms/step - loss: 9094.5400 - val_loss: 19054.0469
Epoch 5/10000
1/1 [==============================] - 0s 16ms/step - loss: 9196.1328 - val_loss: 14964.3311
Epoch 6/10000
1/1 [==============================] - 0s 17ms/step - loss: 7615.5225 - val_loss: 19089.5391
Epoch 7/10000
1/1 [==============================] - 0s 17ms/step - loss: 6819.4521 - val_loss: 16724.0879
Epoch 8/10000
1/1 [==============================] - 0s 17ms/step - loss: 6279.7529 - val_loss: 19287.0156
Epoch 9/10000
1/1 [==============================] - 0s 17ms/step - loss: 6524.2944 - val_loss: 7453.6226
Epoch 10/10000
1/1 [==================

1/1 [==============================] - 0s 16ms/step - loss: 121.4041 - val_loss: 289.3041
Epoch 79/10000
1/1 [==============================] - 0s 16ms/step - loss: 116.6430 - val_loss: 317.3581
Epoch 80/10000
1/1 [==============================] - 0s 17ms/step - loss: 144.7460 - val_loss: 290.4315
Epoch 81/10000
1/1 [==============================] - 0s 16ms/step - loss: 131.5281 - val_loss: 305.1525
Epoch 82/10000
1/1 [==============================] - 0s 16ms/step - loss: 135.2492 - val_loss: 320.7289
Epoch 83/10000
1/1 [==============================] - 0s 16ms/step - loss: 125.7354 - val_loss: 281.1778
Epoch 84/10000
1/1 [==============================] - 0s 16ms/step - loss: 115.5088 - val_loss: 292.5573
Epoch 85/10000
1/1 [==============================] - 0s 17ms/step - loss: 102.2841 - val_loss: 282.0700
Epoch 86/10000
1/1 [==============================] - 0s 16ms/step - loss: 113.0242 - val_loss: 311.8266
Epoch 87/10000
1/1 [==============================] - 0s 16ms/step - l

Epoch 156/10000
1/1 [==============================] - 0s 17ms/step - loss: 108.7116 - val_loss: 231.5687
Epoch 157/10000
1/1 [==============================] - 0s 16ms/step - loss: 89.8060 - val_loss: 186.0269
Epoch 158/10000
1/1 [==============================] - 0s 16ms/step - loss: 100.8809 - val_loss: 189.4267
Epoch 159/10000
1/1 [==============================] - 0s 17ms/step - loss: 164.6051 - val_loss: 353.3632
Epoch 160/10000
1/1 [==============================] - 0s 16ms/step - loss: 135.7434 - val_loss: 229.5851
Epoch 161/10000
1/1 [==============================] - 0s 16ms/step - loss: 121.1161 - val_loss: 217.5867
Epoch 162/10000
1/1 [==============================] - 0s 16ms/step - loss: 101.0489 - val_loss: 201.9841
Epoch 163/10000
1/1 [==============================] - 0s 16ms/step - loss: 110.9080 - val_loss: 171.3270
Epoch 164/10000
1/1 [==============================] - 0s 16ms/step - loss: 101.8914 - val_loss: 118.6861
Epoch 165/10000
1/1 [==========================

1/1 [==============================] - 0s 16ms/step - loss: 57.4145 - val_loss: 15.7688
Epoch 235/10000
1/1 [==============================] - 0s 16ms/step - loss: 50.1899 - val_loss: 15.8234
Epoch 236/10000
1/1 [==============================] - 0s 16ms/step - loss: 47.6153 - val_loss: 8.1680
Epoch 237/10000
1/1 [==============================] - 0s 16ms/step - loss: 51.2829 - val_loss: 48.2463
Epoch 238/10000
1/1 [==============================] - 0s 16ms/step - loss: 67.4920 - val_loss: 42.2068
Epoch 239/10000
1/1 [==============================] - 0s 16ms/step - loss: 62.3873 - val_loss: 31.9685
Epoch 240/10000
1/1 [==============================] - 0s 16ms/step - loss: 61.4879 - val_loss: 12.7491
Epoch 241/10000
1/1 [==============================] - 0s 16ms/step - loss: 56.9967 - val_loss: 19.5124
Epoch 242/10000
1/1 [==============================] - 0s 16ms/step - loss: 59.0993 - val_loss: 18.3585
Epoch 243/10000
1/1 [==============================] - 0s 16ms/step - loss: 49.03

Epoch 313/10000
1/1 [==============================] - 0s 17ms/step - loss: 55.8101 - val_loss: 31.8819
Epoch 314/10000
1/1 [==============================] - 0s 17ms/step - loss: 58.6376 - val_loss: 29.4507
Epoch 315/10000
1/1 [==============================] - 0s 17ms/step - loss: 52.6385 - val_loss: 42.5335
Epoch 316/10000
1/1 [==============================] - 0s 17ms/step - loss: 49.8001 - val_loss: 33.3345
Epoch 317/10000
1/1 [==============================] - 0s 17ms/step - loss: 48.4725 - val_loss: 32.0764
Epoch 318/10000
1/1 [==============================] - 0s 16ms/step - loss: 47.2325 - val_loss: 9.8008
Epoch 319/10000
1/1 [==============================] - 0s 16ms/step - loss: 47.2719 - val_loss: 6.1377
Epoch 320/10000
1/1 [==============================] - 0s 16ms/step - loss: 46.8076 - val_loss: 5.8284
Epoch 321/10000
1/1 [==============================] - 0s 17ms/step - loss: 41.9960 - val_loss: 12.0163
Epoch 322/10000
1/1 [==============================] - 0s 17ms/step

...

1/1 [==============================] - 0s 16ms/step - loss: 4.9658 - val_loss: 0.0214
Epoch 1658/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.7206 - val_loss: 0.0248
Epoch 1659/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.1297 - val_loss: 0.0094
Epoch 1660/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.3178 - val_loss: 0.0470
Epoch 1661/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.7717 - val_loss: 0.0126
Epoch 1662/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.6596 - val_loss: 0.0154
Epoch 1663/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.6163 - val_loss: 0.0349
Epoch 1664/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.5963 - val_loss: 0.0177
Epoch 1665/10000
1/1 [==============================] - 0s 17ms/step - loss: 4.5759 - val_loss: 0.0161
Epoch 1666/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.5342 - val_

1/1 [==============================] - 0s 17ms/step - loss: 3.7628 - val_loss: 0.0132
Epoch 1737/10000
1/1 [==============================] - 0s 17ms/step - loss: 3.7890 - val_loss: 0.0087
Epoch 1738/10000
1/1 [==============================] - 0s 17ms/step - loss: 3.7769 - val_loss: 0.0225
Epoch 1739/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7275 - val_loss: 0.0238
Epoch 1740/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7384 - val_loss: 0.0139
Epoch 1741/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7409 - val_loss: 0.0154
Epoch 1742/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7973 - val_loss: 0.0343
Epoch 1743/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7018 - val_loss: 0.0225
Epoch 1744/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.6912 - val_loss: 0.0263
Epoch 1745/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.6848 - val_

1/1 [==============================] - 0s 16ms/step - loss: 5.4640 - val_loss: 0.0183
Epoch 1816/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.4846 - val_loss: 0.0140
Epoch 1817/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.4132 - val_loss: 0.0217
Epoch 1818/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.4226 - val_loss: 0.0180
Epoch 1819/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.4189 - val_loss: 0.0181
Epoch 1820/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.4013 - val_loss: 0.0184
Epoch 1821/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.3722 - val_loss: 0.0200
Epoch 1822/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.3593 - val_loss: 0.0203
Epoch 1823/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.3428 - val_loss: 0.0213
Epoch 1824/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.3355 - val_

1/1 [==============================] - 0s 16ms/step - loss: 3.7828 - val_loss: 0.0075
Epoch 1895/10000
1/1 [==============================] - 0s 16ms/step - loss: 4.8663 - val_loss: 0.0204
Epoch 1896/10000
1/1 [==============================] - 0s 16ms/step - loss: 6.0194 - val_loss: 0.0940
Epoch 1897/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.5170 - val_loss: 0.0189
Epoch 1898/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7578 - val_loss: 0.0093
Epoch 1899/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.4866 - val_loss: 0.0514
Epoch 1900/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.7760 - val_loss: 0.0082
Epoch 1901/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.7500 - val_loss: 0.0888
Epoch 1902/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.8479 - val_loss: 0.0050
Epoch 1903/10000
1/1 [==============================] - 0s 16ms/step - loss: 5.8584 - val_

1/1 [==============================] - 0s 16ms/step - loss: 3.4976 - val_loss: 0.0409
Epoch 1974/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.4283 - val_loss: 0.0359
Epoch 1975/10000
1/1 [==============================] - 0s 16ms/step - loss: 3.5937 - val_loss: 0.0734
Epoch 1976/10000
1/1 [==============================] - 0s 34ms/step - loss: 3.8182 - val_loss: 0.0145
Epoch 1976: early stopping


In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 54ms/step
[144.989](test_target) - [[42.298916]](prediction) = 102.6900841369629


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,1.846009,0.258449,-1.987494,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,0.603176,1.300254,-1.257479,1.383667,1.255042,-1.083154,-1.554040,-1.803918,-1.803626,-1.697712
1,1.028419,-0.243821,-1.089980,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,0.319231,1.130398,-1.072776,1.147701,1.101670,-0.956314,-1.067021,-0.962149,-1.302342,-1.101641
2,0.747451,-1.326605,-0.362689,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,-0.246181,0.795363,-0.776083,0.765286,0.797618,-0.768352,-0.594770,-0.252717,-0.435038,-0.506657
3,-0.334325,-1.406437,0.198693,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,-0.823243,0.346637,-0.387162,0.289835,0.381344,-0.495113,-0.153504,0.401849,0.313976,0.015627
4,-0.612806,-0.473775,0.598482,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-1.142137,-0.164298,0.074228,-0.225242,-0.108691,-0.112442,0.225831,-0.233874,0.581373,0.333264
5,-0.988232,0.539760,0.840994,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.933047,-0.685960,0.588323,-0.726531,-0.634026,0.403819,0.570985,0.349218,1.072560,0.596153
6,-0.631301,1.100033,0.930545,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,0.073845,-1.166865,1.135363,-1.160620,-1.156201,1.077825,0.848560,0.870549,0.923117,0.736820
7,-1.055214,1.552396,0.871449,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,2.148355,-1.555529,1.695587,-1.474097,-1.636755,1.933732,1.723959,1.631041,0.649980,1.624145


train_target:


,Roraima - Consumo de Cimento (t)
0,39.775
1,30.179
2,34.243
3,35.931
4,60.009
5,63.064
6,86.408
7,103.009


test_input:


...

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
17,0.674508,2.184119,-2.88705,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.268189,-3.467872,1.896178,-1.613363,-1.878166,2.172551,0.943905,-0.349474,-1.517602,0.929344


test_target:


,Roraima - Consumo de Cimento (t)
17,144.989


1/1 [==============================] - 0s 34ms/step
Error: 102.6900841369629




[103.00187683105469,
 99.8343276977539,
 121.89884185791016,
 119.766357421875,
 113.28141021728516,
 105.95062255859375,
 107.4156265258789,
 82.89582824707031,
 119.21044921875,
 42.29891586303711]

In [23]:
display(targets)
display(predictions)

[100.066,
 121.914,
 120.41757142857142,
 113.28205714285713,
 105.48137142857142,
 99.723,
 83.012,
 121.12,
 151.571,
 144.989]

[103.00187683105469,
 99.8343276977539,
 121.89884185791016,
 119.766357421875,
 113.28141021728516,
 105.95062255859375,
 107.4156265258789,
 82.89582824707031,
 119.21044921875,
 42.29891586303711]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

24.46872143859864

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.21065105889411143